# Azure AI Foundry Agent Tracing

## Introduction

This notebook demonstrates **Azure AI Foundry's built-in automatic tracing** for AI agents using OpenTelemetry and Azure Monitor. Tracing provides complete observability into agent behavior, performance, and interactions with minimal code changes.

### What is Agent Tracing?

Agent tracing captures detailed telemetry about:
- **Agent Operations**: Creation, configuration, execution lifecycle
- **LLM Interactions**: Model requests, responses, token usage, latency
- **Tool/Function Calls**: Arguments, results, execution time
- **Message Flow**: User queries, assistant responses, conversation context
- **Performance Metrics**: Latency, success rates, error tracking
- **Cost Analysis**: Token consumption and API usage patterns

All traces automatically flow to **Application Insights** for centralized monitoring and analysis.

### Key Features

**Zero Manual Instrumentation:**
- Azure AI Foundry SDK automatically instruments all agent operations
- No need to manually wrap code with telemetry spans
- Built-in integration with OpenTelemetry standards

**Automatic Export:**
- Traces flow directly to Azure Monitor (Application Insights)
- Real-time visibility into agent performance
- Distributed tracing across services

**Content Recording (Opt-in):**
- Capture full prompts and completions for debugging
- PII-aware configuration options
- Control via environment variables

**Rich Analytics:**
- Query traces with KQL (Kusto Query Language)
- Pre-built dashboards in Azure AI Foundry Portal
- Custom visualizations in Application Insights

**Production-Ready:**
- Built-in retry and error handling
- Buffered export with automatic batching
- Minimal performance overhead

### Tracing Workflow

```
1. Configure OpenTelemetry + Azure Monitor
   ↓
2. Run Agent Operations (automatically traced)
   ↓
3. Traces Export to Application Insights
   ↓
4. View in Azure AI Foundry Portal / App Insights
   ↓
5. Query with KQL for Analysis
   ↓
6. Set Up Alerts & Dashboards
```

### What Gets Automatically Traced

**Agent SDK Operations:**
- `agents.create_agent()` - Agent creation with model config
- `threads.create()` - Conversation thread initialization
- `messages.create()` - User/assistant message creation
- `runs.create_and_process()` - Complete agent execution
- `runs.submit_tool_outputs()` - Function result submission
- `messages.list()` - Message retrieval

**LLM Details:**
- Model name and version
- Prompt tokens, completion tokens, total tokens
- Request/response latency
- Temperature, max_tokens, and other parameters
- Success/failure status codes

**Tool/Function Calls:**
- Function name and description
- Input arguments (JSON)
- Return values
- Execution duration
- Error messages if failures occur

### Benefits

✅ **Complete Visibility**: Every agent operation is traced end-to-end  
✅ **Performance Monitoring**: Identify slow operations and bottlenecks  
✅ **Error Tracking**: Automatic exception capture with stack traces  
✅ **Cost Management**: Track token usage and API costs  
✅ **Debugging**: Reproduce issues with full conversation context  
✅ **Compliance**: Audit trail for all agent interactions  

### Prerequisites

- Azure AI Foundry project with Application Insights enabled
- Azure OpenAI deployment (GPT-4, GPT-4o, or similar)
- Azure credentials configured (`az login`)
- Environment variables configured (see Part 1)

## Table of Contents

1. [Part 1: Environment Setup](#part-1-environment-setup)
   - 1.1: Install Dependencies
   - 1.2: Configure Environment Variables
   - 1.3: Initialize Tracing
2. [Part 2: Trace Agent Creation](#part-2-trace-agent-creation)
3. [Part 3: Trace Agent Execution](#part-3-trace-agent-execution)
4. [Part 4: Trace Function Calling](#part-4-trace-function-calling)
5. [Part 5: View Traces in Azure Portal](#part-5-view-traces-in-azure-portal)
6. [Part 6: Monitoring Best Practices](#part-6-monitoring-best-practices)
7. [Summary and Best Practices](#summary-and-best-practices)

---

## Part 1: Environment Setup

### 1.1: Install Dependencies

Install required packages for Azure AI Foundry tracing with OpenTelemetry and Azure Monitor.

In [ ]:
%pip install -qU --upgrade pip
%pip install -qU pandas
%pip install -qU azure-identity
%pip install -qU azure-ai-projects
%pip install -qU azure-monitor-opentelemetry

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

### 1.2: Configure Environment Variables

Load environment configuration and enable content recording for detailed tracing.

**Required Environment Variables:**
- `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI Foundry project endpoint
- `AZURE_OPENAI_ENDPOINT_GPT_4o`: Azure OpenAI endpoint

**Content Recording:**
- Set `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true` to capture prompts/responses
- ⚠️ May contain personal data - review your compliance requirements

In [ ]:
from dotenv import load_dotenv 
import os
import sys
from pathlib import Path

# Load environment variables
load_dotenv("../.env")

# Add agent utilities to path
current_dir = Path.cwd()
sys.path.insert(0, str(current_dir.parent / '01_agent'))

# Enable content recording for tracing (may contain personal data)
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

# Verify environment variables
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_OPENAI_ENDPOINT_GPT_4o"]
missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    print(f"⚠️  Warning: Missing environment variables: {missing}")
else:
    print("✅ Environment variables loaded successfully")
    print("✅ Content recording enabled for tracing")

### 1.3: Initialize Tracing

Configure Azure Monitor with Application Insights connection string and suppress verbose logging.

**Tracing Setup:**
1. Initialize Azure AI Project Client
2. Retrieve Application Insights connection string
3. Configure Azure Monitor OpenTelemetry exporter
4. Get tracer for custom spans (optional)

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
from opentelemetry.trace import SpanKind, Status, StatusCode
import time
import json
import logging

# Suppress verbose logging
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)
logging.getLogger("azure.identity").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("opentelemetry").setLevel(logging.WARNING)

# Initialize Azure AI Project Client
project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=os.getenv("AZURE_AI_PROJECT_ENDPOINT")
)

print("✅ Azure AI Foundry client initialized")

# Configure Azure Monitor with Application Insights
try:
    connection_string = project_client.telemetry.get_application_insights_connection_string()
    configure_azure_monitor(connection_string=connection_string)
    print("✅ Azure Monitor tracing configured")
    print(f"   Connection string: {connection_string[:50]}...")
except Exception as e:
    print(f"⚠️  Could not configure Azure Monitor: {e}")
    print("   Traces will only be available locally")

# Get tracer for custom spans
tracer = trace.get_tracer(__name__)

print("✅ OpenTelemetry tracing ready")
print("\n📊 Traces will be sent to:")
print("   - Azure Monitor (Application Insights)")
print("   - View in Azure AI Foundry Portal -> Tracing")

---

## Part 2: Trace Agent Creation

Capture telemetry when creating a new agent, including model selection, instructions, and configuration.

**What Gets Traced Automatically:**
- Agent creation API call (`agents.create_agent()`)
- Model configuration (name, deployment)
- Agent metadata (name, instructions, tools)
- Creation latency and success status

**Custom Span Attributes:**
- Add workflow-level context with custom spans
- Track business logic and high-level operations
- Correlate multiple Azure AI operations

In [ ]:
def traced_create_agent(project_client: AIProjectClient, model: str, name: str, instructions: str):
    """Create an agent with Azure AI Foundry built-in tracing"""
    
    with tracer.start_as_current_span(
        "create_agent_workflow",
        kind=SpanKind.CLIENT,
        attributes={
            "agent.name": name,
            "agent.model": model,
            "workflow.type": "agent_creation"
        }
    ) as span:
        start_time = time.time()
        
        try:
            # Azure AI Foundry automatically traces this call
            agent = project_client.agents.create_agent(
                model=model,
                name=name,
                instructions=instructions
            )
            
            latency = time.time() - start_time
            
            # Add workflow-level attributes
            span.set_attribute("agent.id", agent.id)
            span.set_attribute("workflow.latency_ms", latency * 1000)
            span.set_attribute("workflow.success", True)
            span.set_status(Status(StatusCode.OK))
            
            print(f"✅ Agent created: {agent.id}")
            print(f"   Name: {name}")
            print(f"   Model: {model}")
            print(f"   Latency: {latency*1000:.2f}ms")
            
            return agent
            
        except Exception as e:
            span.set_status(Status(StatusCode.ERROR, str(e)))
            span.record_exception(e)
            print(f"❌ Error creating agent: {e}")
            raise

# Example: Create a traced agent
test_agent = traced_create_agent(
    project_client=project_client,
    model="gpt-4o",
    name="TracedTestAgent",
    instructions="You are a helpful assistant for testing tracing capabilities."
)

print("\n💡 View traces in Azure AI Foundry Portal:")
print("   Project → Tracing → Filter by agent ID")

---

## Part 3: Trace Agent Execution

Monitor complete agent runs including thread creation, message processing, and response generation.

**Automatically Traced Operations:**
- Thread creation (`threads.create()`)
- Message creation (`messages.create()`)
- Agent run execution (`runs.create_and_process()`)
- Message retrieval (`messages.list()`)
- Token usage and latency for each step

**Trace Hierarchy:**
```
agent_conversation_workflow (custom)
  ├─ threads.create (auto)
  ├─ messages.create (auto)
  ├─ runs.create_and_process (auto)
  │   ├─ chat.completions (auto)
  │   └─ tool calls (if any)
  └─ messages.list (auto)
```

In [ ]:
def traced_run_agent(project_client: AIProjectClient, agent_id: str, user_message: str):
    """Run an agent with Azure AI Foundry built-in tracing"""

    with tracer.start_as_current_span(
        "agent_conversation_workflow",
        kind=SpanKind.CLIENT,
        attributes={
            "agent.id": agent_id,
            "workflow.type": "agent_execution"
        }
    ) as workflow_span:
        start_time = time.time()

        try:
            # Create thread - automatically traced
            thread = project_client.agents.threads.create()
            workflow_span.set_attribute("thread.id", thread.id)

            # Add message - automatically traced
            project_client.agents.messages.create(
                thread_id=thread.id,
                role="user",
                content=user_message
            )
            workflow_span.set_attribute(
                "message.user_content_length", len(user_message))

            # Run agent - automatically traced with detailed spans
            run = project_client.agents.runs.create_and_process(
                thread_id=thread.id,
                agent_id=agent_id
            )

            workflow_span.set_attribute("run.id", run.id)
            workflow_span.set_attribute("run.status", run.status)

            # Get messages - automatically traced
            messages = list(
                project_client.agents.messages.list(thread_id=thread.id))

            # Extract assistant response
            assistant_message = None
            for msg in messages:
                if msg.role == "assistant":
                    assistant_message = msg.content[0].text.value
                    break

            total_latency = time.time() - start_time

            workflow_span.set_attribute(
                "workflow.latency_ms", total_latency * 1000)
            workflow_span.set_attribute("workflow.success", True)
            if assistant_message:
                workflow_span.set_attribute(
                    "message.assistant_content_length", len(assistant_message))
            workflow_span.set_status(Status(StatusCode.OK))

            print(f"✅ Agent run completed")
            print(f"   Thread ID: {thread.id}")
            print(f"   Run ID: {run.id}")
            print(f"   Status: {run.status}")
            print(f"   Total latency: {total_latency*1000:.2f}ms")
            print(f"\n💬 User: {user_message}")
            print(f"💬 Assistant: {assistant_message}")

            return {
                "thread_id": thread.id,
                "run_id": run.id,
                "status": run.status,
                "user_message": user_message,
                "assistant_message": assistant_message,
                "latency_ms": total_latency * 1000
            }

        except Exception as e:
            workflow_span.set_status(Status(StatusCode.ERROR, str(e)))
            workflow_span.record_exception(e)
            print(f"❌ Error running agent: {e}")
            raise


# Example: Run the traced agent
result = traced_run_agent(
    project_client=project_client,
    agent_id=test_agent.id,
    user_message="What is Azure AI Foundry and its key features?"
)

print("\n💡 View detailed traces in Azure AI Foundry Portal:")
print(f"   Filter by Run ID: {result['run_id']}")
print(f"   Filter by Thread ID: {result['thread_id']}")

---

## Part 4: Trace Function Calling

Monitor agent tool/function calls with automatic instrumentation and custom spans for function execution.

**Function Calling Workflow:**
1. Agent requests function call (traced automatically)
2. Function execution (add custom span for local execution)
3. Submit function results (traced automatically)
4. Agent processes results and generates response (traced automatically)

**Custom Span Best Practices:**
- Wrap local function execution in custom spans
- Add function name, arguments, and results as attributes
- Track execution duration and errors
- Maintain parent-child relationships with workflow span

In [ ]:
# Define example functions
def get_weather(location: str) -> str:
    """Get weather information for a location"""
    return json.dumps({
        "location": location,
        "temperature": "22°C",
        "condition": "Sunny",
        "humidity": "65%"
    })


def calculate_sum(a: float, b: float) -> str:
    """Calculate the sum of two numbers"""
    return str(a + b)


# Function registry for execution
function_registry = {
    "get_weather": get_weather,
    "calculate_sum": calculate_sum
}

# Create agent with STRONG instructions that FORCE function usage
# Based on Microsoft's function calling samples
with tracer.start_as_current_span("create_function_agent_workflow") as span:
    # Use gpt-5 deployment
    model_name = "gpt-5"

    function_agent = project_client.agents.create_agent(
        model=model_name,
        name="FunctionCallingAgent",
        instructions="You must ALWAYS call the provided functions to answer questions. Never answer from your own knowledge.",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "description": "Get current weather for a location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city name (e.g., 'Seattle', 'New York', 'Tokyo')"
                            }
                        },
                        "required": ["location"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "calculate_sum",
                    "description": "Add two numbers together",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "a": {
                                "type": "number",
                                "description": "The first number to add"
                            },
                            "b": {
                                "type": "number",
                                "description": "The second number to add"
                            }
                        },
                        "required": ["a", "b"]
                    }
                }
            }
        ]
    )

    span.set_attribute("agent.id", function_agent.id)
    span.set_attribute("agent.function_count", 2)
    print(f"Agent created: {function_agent.id}")

In [ ]:
print(f"Agent: {function_agent.name}")
print(f"Model: {function_agent.model}")
print(f"Tools: {[t.function.name for t in function_agent.tools]}")

In [ ]:
# Run agent with function calling
with tracer.start_as_current_span("function_calling_workflow") as workflow_span:
    thread = project_client.agents.threads.create()
    user_query = "What's the weather in Seattle? Also, what's 15 + 27?"
    
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_query
    )
    workflow_span.set_attribute("thread.id", thread.id)

    # Create run
    run = project_client.agents.runs.create(
        thread_id=thread.id,
        agent_id=function_agent.id
    )
    workflow_span.set_attribute("run.id", run.id)

    # Process the run with function calling
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Get run status
        run = project_client.agents.runs.get(
            thread_id=thread.id,
            run_id=run.id
        )
        
        # Handle function calls
        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            with tracer.start_as_current_span("execute_functions"):
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    print(f"Calling: {function_name}({function_args})")

                    with tracer.start_as_current_span(
                        function_name,
                        attributes={
                            "function.name": function_name,
                            "function.arguments": json.dumps(function_args)
                        }
                    ) as func_span:
                        func = function_registry[function_name]
                        result = func(**function_args)
                        func_span.set_attribute("function.result", str(result))
                        func_span.set_status(Status(StatusCode.OK))
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": result
                        })
                        print(f"Result: {result}")

            # Submit function results
            project_client.agents.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

    # Get the final response
    messages = project_client.agents.messages.list(thread_id=thread.id)

    print(f"\n🤖 Response:")
    print("=" * 80)
    for msg in messages:
        if msg.role == "assistant":
            if msg.text_messages:
                for text_msg in msg.text_messages:
                    print(text_msg.text.value)
            break
    print("=" * 80)

    workflow_span.set_status(Status(StatusCode.OK))

trace.get_tracer_provider().force_flush()

---

## Part 5: View Traces in Azure Portal

Access and analyze traces through Azure AI Foundry Portal and Application Insights.

**Azure AI Foundry Portal (ai.azure.com):**
- Navigate to your project → Tracing tab
- View agent runs, messages, and tool calls
- Interactive trace hierarchy visualization
- Filter by agent ID, thread ID, or time range
- Performance metrics and success rates

**Application Insights Portal:**
- Comprehensive telemetry dashboard
- Real-time monitoring with Live Metrics
- Custom KQL queries for deep analysis
- Alerts and automated notifications
- Workbooks for custom visualizations

In [ ]:
print("="*80)
print("📊 VIEWING TRACES IN AZURE AI FOUNDRY PORTAL")
print("="*80)

print("\n🌐 Azure AI Foundry Portal:")
print("   1. Navigate to https://ai.azure.com")
print("   2. Select your project")
print("   3. Go to 'Tracing' in the left menu")
print("   4. View traces for all agent operations")

print("\n🔍 What's Automatically Traced:")
print("   ✅ Agent creation and configuration")
print("   ✅ Thread creation and management")
print("   ✅ Message creation (user and assistant)")
print("   ✅ Agent runs and execution")
print("   ✅ Function/tool calls and results")
print("   ✅ Token usage and costs")
print("   ✅ Latency for each operation")
print("   ✅ Errors and exceptions")

print("\n📋 Trace Details Include:")
print("   - Span hierarchy (parent-child relationships)")
print("   - Timestamps and duration")
print("   - Input/output content (if content recording enabled)")
print("   - Model parameters (temperature, max_tokens, etc.)")
print("   - Function arguments and return values")
print("   - Status codes and error messages")

print("\n💡 Application Insights:")
print("   - Traces are also available in Azure Monitor")
print("   - Query using KQL (Kusto Query Language)")
print("   - Set up dashboards and alerts")
print("   - Analyze performance trends")

print("\n" + "="*80)


---

## Part 6: Monitoring Best Practices

Follow these guidelines for effective agent monitoring and observability.

### Content Recording Configuration

**Enable Content Recording:**
```python
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"
```

**Considerations:**
- ⚠️ May capture personal data (PII) in prompts/responses
- Review compliance requirements (GDPR, HIPAA, etc.)
- Consider data residency and retention policies
- Use for debugging/development, disable for production if needed

### Performance Monitoring

**Key Metrics to Track:**
- **Latency**: P50, P95, P99 percentiles for operations
- **Success Rate**: Percentage of successful runs
- **Token Usage**: Prompt/completion tokens per operation
- **Error Rate**: Failed operations and exception types
- **Throughput**: Requests per minute/hour

**Set Up Alerts:**
- Error rate exceeds threshold (e.g., > 5%)
- Average latency exceeds SLA (e.g., > 5 seconds)
- Token usage spikes unexpectedly
- Application Insights connection failures

### Cost Optimization

**Monitor Token Consumption:**
```kql
dependencies
| where name contains "chat.completions"
| extend total_tokens = toint(customDimensions.["gen_ai.usage.total_tokens"])
| summarize total_tokens_per_day = sum(total_tokens) by bin(timestamp, 1d)
```

**Optimization Strategies:**
- Identify prompts with high token counts
- Optimize system instructions for brevity
- Use prompt caching where appropriate
- Monitor and limit max_tokens parameter

### Debugging Workflows

**Use Operation IDs:**
- Every trace has a unique `operation_Id`
- Correlates all spans in a single request
- Use to reconstruct complete conversation flow

**Filter by Agent/Thread:**
```kql
dependencies
| where customDimensions.["gen_ai.agent.id"] == "<agent_id>"
| where customDimensions.["gen_ai.thread.id"] == "<thread_id>"
```

### Production Readiness

**Before Going to Production:**
- ✅ Configure Application Insights retention (default: 90 days)
- ✅ Set up critical alerts (errors, latency, costs)
- ✅ Create operational dashboards for stakeholders
- ✅ Test trace export with production load
- ✅ Document KQL queries for common scenarios
- ✅ Establish baseline metrics for comparison
- ✅ Review content recording compliance

**Cleanup Test Resources:**
- Delete test agents and threads after experiments
- Avoid polluting production telemetry with test data
- Use separate Azure AI projects for dev/staging/prod

In [ ]:
# Cleanup test agents
try:
    if 'test_agent' in locals() and test_agent.id:
        project_client.agents.delete(test_agent.id)
        print(f"✅ Deleted agent: {test_agent.id}")
    
    if 'function_agent' in locals() and function_agent.id:
        project_client.agents.delete(function_agent.id)
        print(f"✅ Deleted agent: {function_agent.id}")
    
    if 'thread' in locals() and thread.id:
        project_client.agents.threads.delete(thread.id)
        print(f"✅ Deleted thread: {thread.id}")
        
    print("\n🧹 Cleanup complete!")
except Exception as e:
    print(f"⚠️ Cleanup warning: {e}")

---

## Summary and Best Practices

### Key Takeaways

1. **Zero-Code Instrumentation**: Azure AI Foundry SDK automatically traces all operations
2. **Complete Observability**: Full visibility into agents, LLMs, and tool calls
3. **Production-Ready**: Built-in export to Application Insights with retry/buffering
4. **Rich Analytics**: KQL queries, dashboards, and alerts in Azure Portal
5. **Content Recording**: Opt-in prompt/response capture for debugging

### What Gets Automatically Traced

**Agent Operations:**
- `agents.create_agent()` - Creation with model config
- `threads.create()` - Thread initialization
- `messages.create()` - User/assistant messages
- `runs.create_and_process()` - Complete execution
- `runs.submit_tool_outputs()` - Function results
- `messages.list()` - Message retrieval

**LLM Interactions:**
- Model name, deployment, version
- Token usage (prompt, completion, total)
- Request/response latency
- Parameters (temperature, max_tokens, etc.)
- Success/failure status codes
- Content (if recording enabled)

**Tool/Function Calls:**
- Function name and description
- Input arguments (JSON)
- Return values
- Execution duration
- Error messages and stack traces

**Context & Metadata:**
- Operation IDs (trace correlation)
- Agent IDs, thread IDs, run IDs
- Custom dimensions and attributes
- Parent-child span relationships
- Timestamps and durations

### Best Practices

#### 1. Tracing Configuration
- ✅ Enable content recording for development/debugging
- ✅ Review compliance before enabling in production (PII concerns)
- ✅ Use Application Insights connection string from Azure AI Project
- ✅ Suppress verbose SDK logging for cleaner output
- ✅ Call `force_flush()` before notebook/script ends

#### 2. Custom Spans (Optional)
- ✅ Add workflow-level spans for business logic
- ✅ Use descriptive span names (e.g., "agent_conversation_workflow")
- ✅ Set `SpanKind.CLIENT` for client operations
- ✅ Add custom attributes for filtering and analysis
- ✅ Record exceptions with `span.record_exception(e)`
- ✅ Set span status: `Status(StatusCode.OK)` or `Status(StatusCode.ERROR)`

#### 3. Performance Monitoring
- ✅ Track latency percentiles (P50, P95, P99)
- ✅ Monitor success rates and error rates
- ✅ Calculate token usage and costs per day/week
- ✅ Identify slow operations for optimization
- ✅ Set up alerts for SLA violations

#### 4. Cost Management
- ✅ Query token usage trends with KQL
- ✅ Identify high-cost prompts and optimize
- ✅ Monitor daily/weekly spending patterns
- ✅ Use prompt caching to reduce repeated calls
- ✅ Set budget alerts in Azure Cost Management

#### 5. Debugging and Troubleshooting
- ✅ Use operation IDs to trace complete conversations
- ✅ Filter by agent ID or thread ID for specific issues
- ✅ Review exception messages and stack traces
- ✅ Check content recording for prompt/response debugging
- ✅ Compare traces before/after changes

#### 6. Production Readiness
- ✅ Configure Application Insights data retention (default: 90 days)
- ✅ Set up critical alerts (errors > 5%, latency > SLA)
- ✅ Create dashboards for stakeholders
- ✅ Test trace export under production load
- ✅ Document common KQL queries
- ✅ Establish baseline metrics
- ✅ Use separate projects for dev/staging/prod
- ✅ Review content recording compliance requirements

### Common KQL Queries Summary

| Query Purpose | Key Fields |
|---------------|-----------|
| All agent operations | `dependencies` table, `name`, `timestamp` |
| Latency analysis | `duration`, `percentile()`, `avg()` |
| Token usage & costs | `gen_ai.usage.total_tokens`, `sum()` |
| Error tracking | `success == false`, `exception.type` |
| Function calls | `gen_ai.tool.name`, `count()` |
| Conversation flows | `operation_Id`, `operation_ParentId` |
| Performance by model | `gen_ai.request.model`, `avg(duration)` |

### Azure Portal Dashboards

**Azure AI Foundry Portal (ai.azure.com):**
- Tracing tab: Agent runs, messages, tool calls
- Performance metrics: Latency, success rates, token usage
- Conversation flows: Interactive trace hierarchy
- Filter & search: By agent, thread, time range

**Application Insights Portal:**
- Application Map: Service topology visualization
- Performance: Latency distribution, slow operations
- Failures: Error rates, exception details
- Live Metrics: Real-time monitoring dashboard
- Workbooks: Custom visualizations with KQL

### Integration Patterns

**CI/CD Quality Gates:**
```yaml
# Example: Check error rate before deployment
- name: Validate Agent Quality
  run: |
    error_rate=$(az monitor app-insights query \
      --analytics-query "dependencies | where timestamp > ago(24h) | summarize error_rate=100.0*countif(success==false)/count()" \
      --output tsv)
    if [ $error_rate -gt 5 ]; then exit 1; fi
```

**Automated Alerts:**
- Error rate > 5% → PagerDuty/Teams notification
- P95 latency > 5s → Email to on-call engineer
- Token usage spikes > 2x baseline → Slack alert
- Application Insights connection failures → Incident

**Dashboard Automation:**
- Daily token usage report → Email stakeholders
- Weekly performance summary → Management dashboard
- Real-time cost tracking → Finance dashboard
- Agent adoption metrics → Product analytics

### Troubleshooting Guide

| Issue | Possible Cause | Solution |
|-------|---------------|----------|
| No traces appearing | App Insights not configured | Verify connection string, check `configure_azure_monitor()` |
| Missing content | Recording disabled | Set `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true` |
| High trace volume | All operations traced | Normal behavior; use KQL filters to focus |
| Traces delayed | Buffering/network latency | Wait 1-2 minutes; call `force_flush()` for immediate export |
| Token fields empty | Not an LLM operation | Only `chat.completions` spans have token data |

### Next Steps

1. **Enable Tracing**: Configure Azure Monitor in your agent code
2. **Run Workloads**: Execute typical agent operations
3. **Explore Portal**: View traces in Azure AI Foundry and App Insights
4. **Create Queries**: Build KQL queries for your use cases
5. **Set Up Alerts**: Configure notifications for critical issues
6. **Build Dashboards**: Create visualizations for stakeholders
7. **Monitor Costs**: Track token usage and optimize prompts
8. **Iterate**: Use insights to improve agent performance

### Additional Resources

- [Azure AI Foundry Tracing Documentation](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-agents-sdk)
- [OpenTelemetry Python SDK](https://opentelemetry.io/docs/instrumentation/python/)
- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-enable)
- [KQL Query Language Reference](https://learn.microsoft.com/azure/data-explorer/kusto/query/)
- [Application Insights Overview](https://learn.microsoft.com/azure/azure-monitor/app/app-insights-overview)

### Related Notebooks

- `01_agent/01_basic_agent.ipynb`: Create and run agents
- `05_evaluation/01_genai_evaluation.ipynb`: Local agent evaluation
- `05_evaluation/05_cloud_based_evaluation.ipynb`: Cloud-based evaluation
- `02_governance/`: Agent governance and compliance